In [1]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate, Lambda
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras import backend as K

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
n_samples = 20000
sample_inputs = []
sample_outputs = []
n_tokens = 61

patterns = [
    lambda i,l: list(range(i, i+l)),
    lambda i,l: list(range(i, i-l, -1))
]

for n in range(n_samples):
    sample_in = []
    sample_out = []
    pattern = np.random.randint(len(patterns))
    for line_number,length in enumerate([5,7,5]):
        i = np.random.randint(10,50)
        #line = list(range(i, i+3))
        line = patterns[pattern](i,length)
        '''if line_number == 0:
            sample_in.append([line[0]] + line)
        else:
            sample_in.append([0] + line)'''
        
        sample_in.append([line[0]] + line + [60]*(9-length))
            
        sample_out.append(line + [60]*(10-length))
        
        
        #sample.append(np.arange(i, i+3))
    #samples.append(sample)
    sample_inputs.append(sample_in)
    sample_outputs.append(sample_out)
    
X = np_utils.to_categorical(sample_inputs, num_classes=n_tokens)
Y = np_utils.to_categorical(sample_outputs, num_classes=n_tokens)

In [13]:
sample_inputs[:10]

[[[39, 39, 40, 41, 42, 43, 60, 60, 60, 60],
  [36, 36, 37, 38, 39, 40, 41, 42, 60, 60],
  [31, 31, 32, 33, 34, 35, 60, 60, 60, 60]],
 [[21, 21, 20, 19, 18, 17, 60, 60, 60, 60],
  [39, 39, 38, 37, 36, 35, 34, 33, 60, 60],
  [25, 25, 24, 23, 22, 21, 60, 60, 60, 60]],
 [[48, 48, 47, 46, 45, 44, 60, 60, 60, 60],
  [22, 22, 21, 20, 19, 18, 17, 16, 60, 60],
  [25, 25, 24, 23, 22, 21, 60, 60, 60, 60]],
 [[32, 32, 33, 34, 35, 36, 60, 60, 60, 60],
  [22, 22, 23, 24, 25, 26, 27, 28, 60, 60],
  [47, 47, 48, 49, 50, 51, 60, 60, 60, 60]],
 [[40, 40, 39, 38, 37, 36, 60, 60, 60, 60],
  [22, 22, 21, 20, 19, 18, 17, 16, 60, 60],
  [42, 42, 41, 40, 39, 38, 60, 60, 60, 60]],
 [[31, 31, 32, 33, 34, 35, 60, 60, 60, 60],
  [18, 18, 19, 20, 21, 22, 23, 24, 60, 60],
  [33, 33, 34, 35, 36, 37, 60, 60, 60, 60]],
 [[43, 43, 42, 41, 40, 39, 60, 60, 60, 60],
  [23, 23, 22, 21, 20, 19, 18, 17, 60, 60],
  [11, 11, 10, 9, 8, 7, 60, 60, 60, 60]],
 [[34, 34, 35, 36, 37, 38, 60, 60, 60, 60],
  [26, 26, 27, 28, 29, 30, 3

In [14]:
latent_dim = 128

# Keras Model

inputs = [Input(shape=(None, n_tokens)) for i in range(3)]
lstms = [LSTM(latent_dim, return_state=True, return_sequences=True) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[lstm_h[i-1], lstm_c[i-1]])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i])
        
denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

# Enforce number of chars per line
counter = Lambda(lambda x: K.sum(K.cast(K.less(K.argmax(x, axis=2), 60), K.floatx()), axis=1, keepdims=True))
counts = [counter(i) for i in outputs]

model = Model(inputs, outputs + counts)

# Use categorical for the character outputs and MSE for the counts
model.compile(optimizer='rmsprop', loss=['categorical_crossentropy']*3 + ['mean_squared_error']*3)
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_11 (InputLayer)                                             (None, None, 61)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_7 (LSTM)                                                     [(None, None, 128), (None, 128), (None, 128 97280                   input_11[0][0]                                                

In [16]:
model.fit(
    [X[:,0,:], X[:,1,:], X[:,2,:]], 
    [
        Y[:,0,:], Y[:,1,:], Y[:,2,:], 
        np.ones(n_samples)*5, np.ones(n_samples)*7,  np.ones(n_samples)*5
    ],
    batch_size=128, epochs=10
)

Epoch 1/10
20000/20000 [==============================] - 7s 359us/step - loss: 0.2045 - dense_7_loss: 0.0852 - dense_8_loss: 0.0777 - dense_9_loss: 0.0050 - lambda_2_loss: 0.0079
Epoch 2/10
20000/20000 [==============================] - 7s 361us/step - loss: 0.1412 - dense_7_loss: 0.0798 - dense_8_loss: 0.0300 - dense_9_loss: 0.0052 - lambda_2_loss: 0.0092
Epoch 3/10
20000/20000 [==============================] - 7s 364us/step - loss: 0.1357 - dense_7_loss: 0.0780 - dense_8_loss: 0.0141 - dense_9_loss: 0.0026 - lambda_2_loss: 0.0089
Epoch 4/10
20000/20000 [==============================] - 7s 364us/step - loss: 0.1015 - dense_7_loss: 0.0758 - dense_8_loss: 0.0094 - dense_9_loss: 0.0037 - lambda_2_loss: 6.0000e-04
Epoch 5/10
20000/20000 [==============================] - 7s 362us/step - loss: 0.1098 - dense_7_loss: 0.0751 - dense_8_loss: 0.0057 - dense_9_loss: 0.0035 - lambda_2_loss: 0.0099
Epoch 6/10
20000/20000 [==============================] - 7s 358us/step - loss: 0.0870 - dense_7

In [17]:
generator_inputs = [Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)) for i in range(3)]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True) for i in range(3)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]

# Hook up line 0
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0]
)
generator_outputs = [generator_denses[0](generator_lstm_out[0])]
generator_models = [Model(generator_inputs[0], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])]

# Set weights for line 0
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())

generator_models[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (1, None, 61)             0         
_________________________________________________________________
lstm_10 (LSTM)               [(1, None, 128), (1, 128) 97280     
_________________________________________________________________
dense_10 (Dense)             (1, None, 61)             7869      
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________


In [18]:
generator_inputs_h = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
generator_inputs_c = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]

#generator_inputs.append(Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)))
#generator_lstms.append(LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True))


# Hook up lines 1 and 2
for i in [1, 2]:
    generator_lstm_out[i], generator_lstm_h[i], generator_lstm_c[i] = generator_lstms[i](
        generator_inputs[i], initial_state=[generator_inputs_h[i], generator_inputs_c[i]]
    )
    generator_outputs.append(generator_denses[i](generator_lstm_out[i]))
#generator_denses.append(Dense(n_tokens, activation='softmax'))

    generator_models.append(Model(
        [generator_inputs[i], generator_inputs_h[i], generator_inputs_c[i]],
        [generator_outputs[i], generator_lstm_h[i], generator_lstm_c[i]]
    ))

    generator_lstms[i].set_weights(lstms[i].get_weights())
    generator_denses[i].set_weights(denses[i].get_weights())

#generator_model_line1.summary()

In [24]:
h,c = None, None
generator_models[0].reset_states()
for i in range(3):
    line = [np.random.randint(10,50)]
    
    def f():
        while True:
            if i > 0:
                yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
            else:
                yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
    while len(line) < 10:
        character, h, c = generator_models[i].predict_generator(f(), steps=1)
        line.append(np.argmax(character))
        
    print(line[1:line.index(60)])

[25, 26, 27, 28, 29]
[38, 39, 40, 41, 42, 43, 44]
[45, 46, 47, 48, 49]
